In [4]:
import os
import cv2
import pandas as pd
from tkinter import filedialog, Tk
from pathlib import Path
from ARMA3_DG_CLASSES import ARMA3_DG_ClassID,ARMA3_DG_vehiclesClasses,ARMA3_DG_ClassNames
import time
import shutil
import tkinter as tk
import numpy as np
import tkinter as tk


In [11]:
def genID():
    return(int(time.time()*10))

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {path}")
    return img

def corners_to_yolo(tl, br, img_width, img_height):
    # Unpack the points
    x_min, y_min = tl
    x_max, y_max = br

    # Calculate center, width, height
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min

    # Normalize values
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    return (x_center, y_center, width, height)

def draw_yoloBB(imgOrig,yolo_bbox,label=''):
    img = np.copy(imgOrig)
    h, w = img.shape[:2]

    # Unpack YOLO values
    _, x_center, y_center, box_w, box_h = yolo_bbox

    # Convert to pixel values
    x_center *= w
    y_center *= h
    box_w *= w
    box_h *= h

    # Convert to top-left and bottom-right corner
    x0 = int(x_center - box_w / 2)
    y0 = int(y_center - box_h / 2)
    x1 = int(x_center + box_w / 2)
    y1 = int(y_center + box_h / 2)

    # Draw rectangle
    cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
    if(label!=''):
        cv2.putText(img,label, (x0, y0-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    return(img)

def yoloBB2Rect(img,yolo_bbox):
    h, w = img.shape[:2]

    # Unpack YOLO values
    _, x_center, y_center, box_w, box_h = yolo_bbox

    # Convert to pixel values
    x_center *= w
    y_center *= h
    box_w *= w
    box_h *= h

    # Convert to top-left and bottom-right corner
    x0 = int(x_center - box_w / 2)
    y0 = int(y_center - box_h / 2)
    x1 = int(x_center + box_w / 2)
    y1 = int(y_center + box_h / 2)

    # Draw rectangle
    cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)

    # Show image
    cv2.imshow("YOLO BBox", img)
    key = cv2.waitKey(250)
    cv2.destroyAllWindows()
    if(key==27):
        return False
    else:
        return True

def get_next_id(directory):
    """
    Get a folder, search for all files with integet as their name.
    Return the next integer in line.
    """
    existing_ids = []
    
    # Scan the directory for image files
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):  # You can add more extensions if needed
            try:
                # Extract the numeric ID from the filename (e.g., "1.jpg", "25.png")
                id_part = filename.split('.')[0]
                existing_ids.append(int(id_part))
            except ValueError:
                continue  # Skip if the filename doesn't have a valid integer ID
    
    # If there are existing IDs, get the next one
    next_id = max(existing_ids, default=0) + 1
    return next_id

def move_file(src, dest):
    """ 
    Given a file 'src', move that file to the correct file specified by 'dest' 
    """
    # exit with error code 4 if this fails...
    img_moved = False
    while not img_moved:
        try:
            shutil.move(src, dest)
        except OSError as e:
            if e.errno == 13:
                # print('File move error caught with OSError.errno 13') #  for debugging
                # This sleep could be 0.2 secs but I don't want to use this high a sleep time.
                # Could be an issue if the user makes the camera rotation time too low
                time.sleep(0.1)
                continue
            # raise the exception if it is not this particular error
            print('ERROR:')
            print('The image file mover process has encountered some unknown error. Please restart the script.')
            print('See the python error description here:')
            print(e)
            exit(4)
        # explicitly raise any other exception type
        except Exception as weirdE:
            print('ERROR:')
            print('The image file mover process has encountered some unknown error. Please restart the script.')
            print('See the python error description here:')
            print(weirdE)
            exit(4)
        img_moved = True

# Function to spawn a window with buttons
def spawn_button_window(keys):
    
    # Create the main window
    root = tk.Tk()
    root.title("Choose an Option")
    root.geometry("300x900")  # Set the window size
    
    ret = 0
    # Function to handle button click
    def on_button_click(choice):
        nonlocal ret
        ret = choice
        root.destroy()
        root.quit()
        
    # Create the specified number of buttons
    for i,key in enumerate(keys):
        button = tk.Button(root, text=key,width=50, command=lambda i=i: on_button_click(i) ,font=("Arial", 20) )
        button.pack(pady=5)  # Add some space between buttons
    
    
    # Start the Tkinter event loop
    root.mainloop()
    return(ret)

def add_sample_to_YOLO_dataset(dtst_path,image_src_path,label):
    if type(label) is tuple:
        label = [label]
    
    dataset_path = Path(dtst_path)
    if(not(((dataset_path/Path("./images/train")).exists()) and
           ((dataset_path/Path("./images/val"  )).exists()) and
           ((dataset_path/Path("./labels/train")).exists()) and
           ((dataset_path/Path("./labels/val")  ).exists()))):
        print("Not a proper dataset structure")
        return False
    train_images_path = dataset_path/Path("./images/train")
    train_labels_path = dataset_path/Path("./labels/train")

    id = str(genID())
    image_path = os.path.join(train_images_path,f"{id}.png")
    label_path = os.path.join(train_labels_path,f"{id}.txt")

    with open(label_path, "w") as f:
        for bb in label:
            print(bb)
            classIdx,x_center,y_center,norm_w,norm_h = bb
            if(classIdx!=-1):
                f.write(f"{classIdx} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")
                
    move_file(image_src_path,image_path)
    print(f"Add::idx {id}::{image_path}")


# Review and move Positive Samples

In [12]:
import os
import cv2
import pandas as pd

def convert_to_yolo(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2.0 / img_width
    y_center = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return (x_center, y_center, width, height)

def interactive_bb_review(folder_path, csv_path):
    df = pd.read_csv(csv_path)
    df.set_index("id", inplace=True)

    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')])
    object_images = [f for f in image_files if ((not 'blank' in f.lower()) and (not 'marked' in f.lower()))]
    object_images.sort()
    print("Number of files: ",len(object_images))
    idx = 0
    reviewed_data = {}

    def select_bb(image):
        bbox = cv2.selectROI("Select ROI (Enter to confirm, Esc to cancel)", image, fromCenter=False, showCrosshair=True)
        cv2.destroyAllWindows()
        if bbox[2] == 0 or bbox[3] == 0:
            return None
        x, y, w, h = bbox
        return (x, y, x + w, y + h)

    while 0 <= idx < len(object_images):
        ID         = int(object_images[idx][:-4])
        obj_name   = object_images[idx]
        blank_name = obj_name.replace('.png', '_blank.png')

        obj_path = os.path.join(folder_path, obj_name)
        blank_path = os.path.join(folder_path, blank_name)

        obj_img = cv2.imread(obj_path)
        blank_img = cv2.imread(blank_path)

        if obj_img is None or blank_img is None:
            print(f"Could not load image pair: {obj_name}, {blank_name}")
            idx += 1
            continue

        h, w = obj_img.shape[:2]
        bbs = []

        if ID in df.index:
            row = df.loc[ID]
            if isinstance(row, pd.Series):
                tl = tuple(map(int,row.TL.strip("()").split(',')))
                br = tuple(map(int,row.BR.strip("()").split(',')))
                cls = ARMA3_DG_ClassID[row['v_class']]
                x_center,y_center,norm_w,norm_h = corners_to_yolo(tl, br, w, h)
                bbs = [(cls,x_center,y_center,norm_w,norm_h)]
            else:
                row = row.iloc[0]
                bbox = (int(row['x_min']), int(row['y_min']), int(row['x_max']), int(row['y_max']))
                cls = ARMA3_DG_ClassID[row['v_class']]
                x_center,y_center,norm_w,norm_h = convert_to_yolo(bbox, w, h)
                bbs = [(cls,x_center,y_center,norm_w,norm_h)]
        else:
            print(f"No BB found for {obj_name} in CSV.")

        while True:
            # Draw current BBs on a copy
            display_img = obj_img.copy()
            for bb in bbs:
                cls,xc, yc, bw, bh = bb
                # Convert back to absolute coords for drawing
                x_min = int((xc - bw / 2) * w)
                y_min = int((yc - bh / 2) * h)
                x_max = int((xc + bw / 2) * w)
                y_max = int((yc + bh / 2) * h)
                cv2.rectangle(display_img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(display_img, ARMA3_DG_ClassNames[cls], (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            key = 0xFF
            while key not in [32,27,ord('e'),ord('q'),ord('d'),ord('a'),13]:  # 32 is the ASCII code for the spacebar
                key = 0xFF
                # Display "with.png" and "wout.png" alternately
                cv2.imshow(f"Image Pair:", display_img)
                key &=cv2.waitKey(1000) & 0xFF  # Wait for 30 ms
                
                cv2.imshow(f"Image Pair:", blank_img)
                key &=cv2.waitKey(100) & 0xFF  # Wait for 30 ms
            cv2.destroyAllWindows()

            skip=False
            if key == 27:  # Skip sample
                idx += 1
                skip=True
                break

            elif key == ord('q'): # Quit
                cv2.destroyAllWindows()
                return reviewed_data
            
            elif key == ord('a'):  # Prev
                idx -= 1
                break

            elif key == ord('d'):  # Next
                idx += 1
                break

            elif key == 32:  # Space = reselect first BB, keep class
                if bbs:
                    print(f"Reselecting BB for class: {ARMA3_DG_ClassNames[bbs[0][0]]}")
                    new_bbox = select_bb(obj_img)
                    if new_bbox:
                        x_center,y_center,norm_w,norm_h = convert_to_yolo(new_bbox, w, h)
                        bbs[0] = (bbs[0][0],x_center,y_center,norm_w,norm_h)

            elif key == ord('e'):  # add Extra bb
                print("Select new BB...")
                new_bbox = select_bb(obj_img)
                if new_bbox:
                    # Get class
                    new_class =spawn_button_window(ARMA3_DG_ClassID.keys())
                    x_center,y_center,norm_w,norm_h = convert_to_yolo(new_bbox, w, h)
                    bbs.append((new_class,x_center,y_center,norm_w,norm_h))
                    
        if(not skip):
            reviewed_data[ID] = bbs
        else:
            reviewed_data.pop(ID,None)
    return reviewed_data


In [13]:
folder_path = r"D:\synthetic_dataset\arma\sea\samples_positive"  # Set your folder path here
csv_path    = r"D:\synthetic_dataset\arma\sea\samples_positive\AnnotationData.csv"
yolo_data_path = r"D:\synthetic_dataset\arma\yolo_arma3_dataset_sea"

reviewed_samples = interactive_bb_review(folder_path,csv_path)

save = input(f"Move {len(reviewed_samples)} images to dataset? (y/n)").lower()
if save == 'y':
    for i in reviewed_samples:
        filename = str(i)+'.png'
        filename2 = str(i)+'_blank.png'
        filename3 = str(i)+'_marked.jpg'

        add_sample_to_YOLO_dataset(yolo_data_path,os.path.join(folder_path, filename),reviewed_samples[i])
        os.remove(os.path.join(folder_path, filename2))
        os.remove(os.path.join(folder_path, filename3))
        time.sleep(0.5)

Number of files:  44
Reselecting BB for class: MIL_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Select new BB...
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Select new BB...
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT
Reselecting BB for class: CIV_BOAT


In [ ]:
def review_positive_and_yolo_labeling(folder,csv_path,yolo_data_path):
    # File selection dialogs
    root = Tk()
    root.withdraw()

    if("Open datasetFolder"):
        dataset_path = Path(yolo_data_path)
        if(not(((dataset_path/Path("./images/train")).exists()) and
                ((dataset_path/Path("./images/val")).exists()) and
                ((dataset_path/Path("./labels/train")).exists()) and
                ((dataset_path/Path("./labels/val")).exists()))):
            print("Not all apropropriate folders exist")
            return

    if("Open arma images folder"):
        df = pd.read_csv(csv_path)


    for row in df.itertuples():
        tl = tuple(map(int,row.TL.strip("()").split(',')))
        br = tuple(map(int,row.BR.strip("()").split(',')))
        base_name = ("{:06d}".format(row[1])) # Assuming the first column has image base names (without postfixes)
        img1_path = os.path.join(folder, f"{base_name}.png")
        img2_path = os.path.join(folder, f"{base_name}_blank.png")
        className = row[8]

        if(not Path(img1_path).exists()):
            continue
        try:
            print("Sample:",base_name,end=' : ',flush=True)
            img1 = load_image(img1_path)
            img2 = load_image(img2_path)
        except FileNotFoundError as e:
            print(e)
            continue
        
        h, w = img1.shape[:2]
        yoloBB = corners_to_yolo(tl, br, w, h)
        
        # Resize all images to same height for display
        if("Open images"):
            height = 800
            def resize_to_height(img):
                h, w = img.shape[:2]
                scale = height / h
                return cv2.resize(img, (int(w * scale), height))

            img1_resized = resize_to_height(img1)
            img2_resized = resize_to_height(img2)

        # Toggle between the images until spacebar is pressed
        labeled = draw_yoloBB(img1_resized,yoloBB,className)
        key = 0xFF
        while key not in [32,27,ord('q'),ord('d'),13]:  # 32 is the ASCII code for the spacebar
            key = 0xFF
            # Display "with.png" and "wout.png" alternately
            cv2.imshow(f"Image Pair:", labeled)
            key &=cv2.waitKey(1000) & 0xFF  # Wait for 30 ms
            
            cv2.imshow(f"Image Pair:", img2_resized)
            key &=cv2.waitKey(100) & 0xFF  # Wait for 30 ms
        cv2.destroyAllWindows()

        if("Parse input"):
            deleting = False
            if key == 32:  ####################################### Press SPACE to review the image
                print("Further processing")
                sampleOK = False
            elif key == ord('q'):  ############################### Press q to exit early
                print("Stop")
                break
            elif key == 27:  ##################################### Press ESC to skip sample
                print("Skip")
                continue
            elif key == ord('d'):  ############################### Press d to delete image set
                print("Deleting sample")
                deleting = True        
            else: ################################################ Enter to store as is
                print("Sample OK")
                sampleOK = True
            
        if(deleting == True):
            print("Delete:")
            print(img1_path)
            print(img2_path)
            print(img3_path)
            emptyPath = Path(r"D:\synthetic_dataset\arma\samples_empty")
            id = genID()
            emptyPath = emptyPath/Path(f"{str(id)}.png")
            move_file(img1_path,emptyPath)
            os.remove(img2_path)
            os.remove(img3_path)
            continue

        if sampleOK:
            print("Keeping the defined BB.")
            x = tl[0]
            y = tl[1]
            w = br[0]-x
            h = br[1]-y
            img_h, img_w = img1.shape[:2]
        else:
            print(f"Select ROI for {base_name}...")
            roi = cv2.selectROI("Select ROI on Left Image", img1_resized, showCrosshair=True)
            cv2.destroyWindow("Select ROI on Left Image")
            if roi == (0, 0, 0, 0):
                print("No ROI skipping image")
                continue
            else:
                x, y, w, h = roi
                img_h, img_w = img1_resized.shape[:2]
    
        # Normalize to YOLO format: x_center, y_center, width, height (all 0-1)
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        norm_w = w / img_w
        norm_h = h / img_h

        if(sampleOK):
            save = True
        else:
            save = yoloBB2Rect(img1,(0,x_center,y_center,norm_w,norm_h))
            if(not save):
                continue

        add_sample_to_YOLO_dataset(dataset_path,img1_path,(ARMA3_DG_ClassID[className],x_center,y_center,norm_w,norm_h))
        os.remove(img2_path)
        os.remove(img3_path)

        cv2.waitKey(1)
        cv2.destroyAllWindows()


    cv2.destroyAllWindows()
    print("Done.")


In [6]:
if __name__ == "__main__":
    # Example usage
    folder_path = r"D:\synthetic_dataset\arma\sea\samples_positive"  # Set your folder path here
    csv_path    = r"D:\synthetic_dataset\arma\sea\samples_positive\AnnotationData.csv"
    yolo_data_path = r"D:\synthetic_dataset\arma\yolo_arma3_dataset_sea"
    review_positive_and_yolo_labeling(folder_path,csv_path,yolo_data_path)


Sample: 17447254463 : Skip
Sample: 17447254652 : Stop
Done.



# Manual labeling of negative images 

In [13]:
def manual_sigle_yolo_labeling(folder_path,yolo_data_path):
    # Get the list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(".png") or f.endswith(".jpg")]
    from random import shuffle

    shuffle(image_files)
    for img in image_files:
        im_path = os.path.join(folder_path, img)
        image   = cv2.imread(im_path)

        labels = []
        while(1):      
            # Once spacebar is pressed, show the "with.png" image for ROI selection
            roi = cv2.selectROI(f"Select BB", image)
            
            # Crop the selected ROI
            if roi != (0, 0, 0, 0):
                # Normalize to YOLO format: x_center, y_center, width, height (all 0-1)
                x, y, w, h = roi
                img_h, img_w = image.shape[:2]
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                norm_w = w / img_w
                norm_h = h / img_h

                # Get class
                classIDX = spawn_button_window(ARMA3_DG_ClassID.keys())
                # Draw rectangle
                cv2.rectangle(image, (roi[0], roi[1]), (roi[0]+roi[2], roi[1]+roi[3]), (0, 255, 0), 2)
                cv2.imshow(f"Selected ROI", image)
                key=cv2.waitKey(0)  # Wait until a key is pressed to close the image window
                if(key == 27):
                    pass
                if(key == ord('q')):
                    cv2.destroyAllWindows()
                    return
                else:
                    labels.append((classIDX,x_center,y_center,norm_w,norm_h))
                cv2.destroyAllWindows()
                
            else:
                labels.append((-1,-1,-1,-1,-1))
                break
        # Close the OpenCV window
        cv2.destroyAllWindows()

        add_sample_to_YOLO_dataset(yolo_data_path,im_path,labels)

# Example usage
folder_path = r"D:\synthetic_dataset\arma\samples_negative"  # Set your folder path here
yolo_data_path = r"D:\synthetic_dataset\arma\yolo_arma3_dataset"
manual_sigle_yolo_labeling(folder_path,yolo_data_path)


(-1, -1, -1, -1, -1)
Add::idx 17436154372::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154372.png
(-1, -1, -1, -1, -1)
Add::idx 17436154380::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154380.png
(-1, -1, -1, -1, -1)
Add::idx 17436154392::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154392.png
(-1, -1, -1, -1, -1)
Add::idx 17436154398::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154398.png
(-1, -1, -1, -1, -1)
Add::idx 17436154406::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154406.png
(-1, -1, -1, -1, -1)
Add::idx 17436154412::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154412.png
(-1, -1, -1, -1, -1)
Add::idx 17436154422::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154422.png
(-1, -1, -1, -1, -1)
Add::idx 17436154428::D:\synthetic_dataset\arma\yolo_arma3_dataset\images\train\17436154428.png
(-1, -1, -1, -1, -1)
Add::idx 17436154435::D:\synthetic_dataset\

# Manual labeling  inconclusive samples

In [64]:
def manual_pair_yolo_labeling(folder_path,yolo_data_path):
    """"Go through a folder ad find pairs with postfix with.png, wout.png.
    - Choose  input folder and dataset folder
    - Space to continue and choose BB
    - Esc to kip image
    """
    # Get the list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith("with.png") or f.endswith("wout.png")]
    
    # Sort files to ensure that "with.png" and "wout.png" are paired
    image_pairs = {}
    for img in image_files:
        prefix = "-".join(img.split("-")[:-1])
        if prefix not in image_pairs:
            image_pairs[prefix] = {}
        if img.endswith("with.png"):
            image_pairs[prefix]["with"] = img
        elif img.endswith("wout.png"):
            image_pairs[prefix]["wout"] = img
    
    # Loop through each pair of images
    for prefix, pair in image_pairs.items():
        filename_tokens = prefix.split('-')
        className = ARMA3_DG_vehiclesClasses[filename_tokens[1]]
        classIDX  = ARMA3_DG_ClassID[className]
        
        with_image_path = os.path.join(folder_path, pair["with"])
        wout_image_path = os.path.join(folder_path, pair["wout"])
        
        # Read both images
        with_image = cv2.imread(with_image_path)
        wout_image = cv2.imread(wout_image_path)
        
        # Check if images are loaded successfully
        if with_image is None or wout_image is None:
            print(f"Error loading images: {with_image_path} or {wout_image_path}")
            continue
        
        # Toggle between the images until spacebar is pressed
        key = 0xFF
        while key != 32 and key!= 27 and key != ord('d'):  # 32 is the ASCII code for the spacebar
            key = 0xFF
            # Display "with.png" and "wout.png" alternately
            cv2.imshow(f"Image Pair: {prefix}", with_image)
            key &=cv2.waitKey(100) & 0xFF  # Wait for 30 ms
            
            cv2.imshow(f"Image Pair: {prefix}", wout_image)
            key &=cv2.waitKey(100) & 0xFF  # Wait for 30 ms
        cv2.destroyAllWindows()
        
        if(key==27):
            continue

        if(key==ord('d')):
            os.remove(with_image_path)
            os.remove(wout_image_path)
            continue

        while(1):      
            # Once spacebar is pressed, show the "with.png" image for ROI selection
            roi = cv2.selectROI(f"Select ROI for {prefix}", with_image)
            
            # Crop the selected ROI
            if roi != (0, 0, 0, 0):
                # Normalize to YOLO format: x_center, y_center, width, height (all 0-1)
                x, y, w, h = roi
                img_h, img_w = with_image.shape[:2]
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                norm_w = w / img_w
                norm_h = h / img_h
                label = (classIDX,x_center,y_center,norm_w,norm_h)
                # Draw rectangle
                cv2.rectangle(with_image, (roi[0], roi[1]), (roi[0]+roi[2], roi[1]+roi[3]), (0, 255, 0), 2)
                cv2.imshow("Confirm with Space", with_image)
                key=cv2.waitKey(0)  # Wait until a key is pressed to close the image window
                if(key == 32):
                    break
                cv2.destroyAllWindows()
                
            else:
                label = (-1,-1,-1,-1,-1)
                break
        # Close the OpenCV window
        cv2.destroyAllWindows()

        add_sample_to_YOLO_dataset(yolo_data_path,with_image_path,label)
        if(label[0]>=0):
            add_sample_to_YOLO_dataset(yolo_data_path,wout_image_path,(-1,-1,-1,-1,-1))
        else:
            os.remove(wout_image_path)
# Example usage
folder_path = r"D:\synthetic_dataset\arma\samples_inconclusive"  # Set your folder path here
yolo_data_path = r"D:\synthetic_dataset\arma\yolo_arma3_dataset"
manual_pair_yolo_labeling(folder_path,yolo_data_path)


# Revisit dataset to fill in missing labels

In [10]:
def revisit_samples(image_folder, label_folder, classes = ARMA3_DG_ClassID,onlyClassed = True):
    """
    Go through all samples to mark unlabeled images.
    If nothing to labe, press any  button to exit SelectROI window and move on.
    if sothing you selected is not ok press ESC to start again.
    """
    from random import shuffle
    # Get list of all image files
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
    shuffle(image_files)
    for image_file in image_files:
        # Get the corresponding label file (assuming the same name)
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(label_folder, label_file)

        # Read the image
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)


        # If label file exists, read the bounding boxes
        if os.path.exists(label_path):
            classedIm = False
            with open(label_path, 'r') as f:
                for line in f:
                    classedIm = True
                    # YOLO format: class_num x_center y_center width height (normalized)
                    class_num, x_center, y_center, width, height = map(float, line.strip().split())
                    # Convert normalized values back to pixel values
                    img_height, img_width = image.shape[:2]
                    x1 = int((x_center - width / 2) * img_width)
                    y1 = int((y_center - height / 2) * img_height)
                    x2 = int((x_center + width / 2) * img_width)
                    y2 = int((y_center + height / 2) * img_height)
                    # Draw existing bounding boxes
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
                    cv2.putText(image,ARMA3_DG_ClassNames[class_num], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)


            if((not classedIm) and onlyClassed):
                continue

            while(1):
                # Once spacebar is pressed, show the "with.png" image for ROI selection
                roi = cv2.selectROI(f"Select BB {image_file}", image)
                
                # Crop the selected ROI
                if roi != (0, 0, 0, 0):
                    # Normalize to YOLO format: x_center, y_center, width, height (all 0-1)
                    x, y, w, h = roi
                    img_h, img_w = image.shape[:2]
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    norm_w = w / img_w
                    norm_h = h / img_h

                    # Get class
                    classIDX = spawn_button_window(ARMA3_DG_ClassID.keys())
                    # Draw rectangle
                    temp = np.copy(image)
                    cv2.rectangle(temp, (roi[0], roi[1]), (roi[0]+roi[2], roi[1]+roi[3]), (0, 0, 255), 1)
                    cv2.putText(temp,ARMA3_DG_ClassNames[classIDX], (roi[0], roi[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    cv2.imshow(f"New BB Esc:Pass,Q,Space", temp)
                    key=cv2.waitKey(0)  # Wait until a key is pressed to close the image window
                    if(key == 27):
                        pass
                    elif(key==ord('q')):
                        cv2.destroyAllWindows()
                        return
                    elif(key==32): #Space bar
                        # Append the new bounding box (ROI) to the label file
                        cv2.rectangle(image, (roi[0], roi[1]), (roi[0]+roi[2], roi[1]+roi[3]), (0,255, 0), 1)
                        cv2.putText(image,ARMA3_DG_ClassNames[classIDX], (roi[0], roi[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                        with open(label_path, 'a') as f:
                            f.write(f"{classIDX} {x_center} {y_center} {norm_w} {norm_h}\n")
                    cv2.destroyAllWindows()
                else:
                    break


        # Close the OpenCV window
        cv2.destroyAllWindows()

# Example usage
image_folder = r'D:\synthetic_dataset\arma\yolo_arma3_dataset_sea\images\train'
label_folder = r'D:\synthetic_dataset\arma\yolo_arma3_dataset_sea\labels\train'
revisit_samples(image_folder, label_folder)

KeyboardInterrupt: 

# Perform train/validation split

In [14]:
import os
import shutil
import random
from pathlib import Path

def balance_train_val_split(dataset_path, image_ext=".jpg", train_ratio=0.8):
    """
    Balances the train/val image and label folders to match the specified train_ratio (default: 80/20).
    
    Args:
        dataset_path (str or Path): Root dataset directory containing 'images/train', 'images/val',
                                    'labels/train', and 'labels/val' subfolders.
        image_ext (str): Extension of image files (default: '.jpg').
        train_ratio (float): Desired ratio of training data (default: 0.8 for 80/20 split).
    """
    dataset_path = Path(dataset_path)
    images_path = dataset_path / "images"
    labels_path = dataset_path / "labels"

    train_images = list((images_path / "train").glob(f"*{image_ext}"))
    val_images = list((images_path / "val").glob(f"*{image_ext}"))

    n_train = len(train_images)
    n_val = len(val_images)
    total = n_train + n_val

    print(f"[INFO] Current split: {n_train} train / {n_val} val (total: {total})")

    target_train = int(total * train_ratio)
    target_val = total - target_train

    diff_train = n_train - target_train
    diff_val = n_val - target_val

    def move_files(files, src_img_dir, dst_img_dir, src_lbl_dir, dst_lbl_dir, count):
        selected = random.sample(files, count)
        for img_path in selected:
            label_name = f"{img_path.stem}.txt"
            label_path = src_lbl_dir / label_name

            shutil.move(str(img_path), dst_img_dir / img_path.name)
            if label_path.exists():
                shutil.move(str(label_path), dst_lbl_dir / label_name)

    if diff_train > 0:
        print(f"[ACTION] Moving {diff_train} images from train → val to balance.")
        move_files(train_images, images_path / "train", images_path / "val",
                   labels_path / "train", labels_path / "val", diff_train)

    elif diff_val > 0:
        print(f"[ACTION] Moving {diff_val} images from val → train to balance.")
        move_files(val_images, images_path / "val", images_path / "train",
                   labels_path / "val", labels_path / "train", diff_val)
    else:
        print("[OK] Dataset already balanced.")

    # Final count
    final_train = len(list((images_path / "train").glob(f"*{image_ext}")))
    final_val = len(list((images_path / "val").glob(f"*{image_ext}")))
    print(f"[DONE] Final split: {final_train} train / {final_val} val")

#Example usage
balance_train_val_split(r"D:\synthetic_dataset\arma\yolo_arma3_dataset_sea", image_ext=".png", train_ratio=0.8)

[INFO] Current split: 498 train / 117 val (total: 615)
[ACTION] Moving 6 images from train → val to balance.
[DONE] Final split: 492 train / 123 val
